# LlamaIndexでRAG

## 目次
- [概要](#概要)
- [参考](#参考)
- [チェック](#チェック)
- [チュートリアル](#チュートリアル)
  - [準備](#準備)
    - [使用する変数](#使用する変数)
    - [インストレーション](#インストレーション)
    - [ライブラリ読み込み](#ライブラリ読み込み)
  - [LLM](#LLM)
    - [OpenAI](#OpenAI)
      - [追加のインストレーション](#追加のインストレーション1)
      - [追加のライブラリ読み込み](#追加のライブラリ読み込み1)
      - [API Keyの確認](#API_Keyの確認)
    - [Ollama](#Ollama)
      - [追加のインストレーション](#追加のインストレーション2)
      - [追加のライブラリ読み込み](#追加のライブラリ読み込み2)
      - [追加のライブラリの設定](#追加のライブラリの設定)
  - [共通](#共通)
    - [インデックス作成](#インデックス作成)
    - [RAGのRetrieval部](#RAGのRetrieval部)
    - [ログの有効化](#ログの有効化)
    - [永続化して実行](#永続化して実行)
- [詳細](#詳細)
  - [Queryingのカスタマイズ](#Queryingのカスタマイズ)
    - LLMに投げる前にチャンクを確認
    - 段階毎のカスタマイズ
  - [Evaluation](#Evaluation)
    - 準備
    - 関連評価
    - 回答評価
    - 検索評価
  - [StoringでChromaDBを使用](#StoringでChromaDBを使用)
    - 追加のインストレーション3
    - 追加のライブラリ読み込み3
    - パーツ毎に分解して実行

## 概要
- LlamaIndex（公式）をトレースして基本的な利用方法を確認する。
- 破壊的に変更が発生するまで使えるでしょう。
- 破壊的に変更が発生後は、公式サイトの当該バージョンの情報（≒一次情報）をあたって。

## 参考

LLMのRAG - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LLM%E3%81%AERAG
- 知識情報の分割
- 知識情報の埋め込み
- 質問の入力（Query Input）
- 質問の埋め込み（Query Embedding）
- 情報の検索（Information Retrieval）
- 情報の生成（Information Generation）
- 回答の提供（Answer Delivery）

LlamaIndex - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LlamaIndex
- Loading
- Indexing
- Storing
- Querying
- Evaluation

## チェック

In [1]:
#!pip list

In [2]:
#%env

## チュートリアル

### 準備

#### 使用する変数

In [3]:
DATA_DIR = "./llamaindex/data/paul_graham_essay"
PERSIST_DIR = "./llamaindex/storage/paul_graham_essay"
CHROMA_DIR = "./llamaindex/chroma_db/paul_graham_essay"

#### インストレーション

```bash
!pip install llama-index
```

#### ライブラリ読み込み

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

### LLM

#### OpenAI

##### 追加のインストレーション1

In [5]:
# 不要（OpenAIは依存関係パッケージらしい）

##### 追加のライブラリ読み込み1

In [6]:
# 評価の所で使う
from llama_index.llms.openai import OpenAI

##### API_Keyの確認
- 準備は、OpenAIにログインしてAPIからKeyを取得、カード登録してチャージするだけ。
- プログラムから使用できるようにするには、以下の環境変数に、API Keyが設定されていれば良い。
- 冷静に考えると、評価の所でLLMを明示的に指定しているがそれ以外でなんのモデルを使っているか不明。

```Python
import os
print(os.environ['OPENAI_API_KEY'])
```

#### Ollama

##### 追加のインストレーション2

```bash
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-huggingface
```

##### 追加のライブラリ読み込み2

```Python
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
```

##### 追加のライブラリの設定

```Python
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = Ollama(model="Llama3", request_timeout=360.0)
```

### 共通
以降のコードは、OpenAI・Ollamaで共通になる。  
（ただし、評価で使うLLMにはOpenAIを使用）

#### インデックス作成
最も基礎的で、オンメモリのセマンティック検索のインデックス。
- RAGで言うと：知識情報の分割、知識情報の埋め込み
- LlamaIndexで言うと：Loading、Indexing

##### Loading

In [7]:
documents = SimpleDirectoryReader(DATA_DIR).load_data()

##### Indexing
- 以下では、VectorStoreIndexを使用している。
- SummaryIndexやKnowledgeGraphIndexなどのindexもある。
- 質問内容によって最適なIndexが異なる可能性がある。

In [8]:
index = VectorStoreIndex.from_documents(
    documents,
)

#### RAGのRetrieval部
- RAGで言うと：Query Input、Query Embedding、Information Retrieval
- LlamaIndexで言うと：Querying、Evaluation

##### Querying

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

The author worked on writing short stories and programming, starting with early attempts on an IBM 1401 in 9th grade using an early version of Fortran. Later, the author transitioned to working with microcomputers, building simple games and programs on a TRS-80.


#### ログの有効化
抽象化度が高いので、ログの有効化をしても良いが、結局、欲しい所が出きってない感もある。

```Python
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
```

#### 永続化して実行
前述のコードに、LlamaIndexで言うStoringの概念を追加したIndexing & Storing版。
- 永続化は、Document Store、Vector Store、Index Storeに、Storage Contextを設定する。
- 既出の、Loadingの所で、Document Storeから読み出している。
- Indexingでは、Vector Store、Index Storeに書き出し（永続化し）す。

OpenAI・Ollamaを切り替える際は、Embeddingが異なるようなので、一度、PERSIST_DIRのIndexを削除する。

In [10]:
import os.path
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(DATA_DIR).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

The author worked on writing short stories and programming, particularly on an IBM 1401 in 9th grade using an early version of Fortran. Later, the author got a TRS-80 computer and started programming more extensively, creating simple games, a rocket prediction program, and a word processor.


## 詳細

### Queryingのカスタマイズ

#### LLMに投げる前にチャンクを確認

#### no_textを使う
オプションでLLMにリクエストしない方法がある。

In [11]:
# Either way we can now query the index
query_engine = index.as_query_engine(response_mode='no_text')
response = query_engine.query("What did the author do growing up?")
# print(response.source_nodes) # プロパティを確認したい場合、ココを実行して出力を確認

#### as_retrieverを使う
そもそも、別メソッドが用意されているらしい。

In [12]:
retriever = index.as_retriever(similarity_top_k=5)
nodes = retriever.retrieve("What did the author do growing up?")
# print(response.source_nodes) # プロパティを確認したい場合、ココを実行して出力を確認

#### MockLLMを使う
MockLLMを使うという方法もあるらしい。

```Python
from llama_index.core.llms import MockLLM
llm = MockLLM()
```

#### source_nodesを確認する

In [13]:
for node in response.source_nodes:
    print("score ", node.score)
    print("id_", node.id_)
    print("file_name", node.metadata["file_name"])
    # print("text", node.text) # テキストを確認したい場合、ココを実行して出力を確認
    print("------------------------------------------------------")

score  0.8108766833727953
id_ ef903a82-4999-448a-b007-452ab00100d5
file_name paul_graham_essay.txt
------------------------------------------------------
score  0.804940970839821
id_ 4aa4469e-4ece-4418-94ac-3a824149c490
file_name paul_graham_essay.txt
------------------------------------------------------


#### 段階毎のカスタマイズ
クエリによる検索、後処理、応答合成の各段階をカスタマイズできる。

##### 検索
K=3検索

In [14]:
from llama_index.core.retrievers import VectorIndexRetriever

# configure retriever
# トップ「10」セマンティック検索
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

##### 後処理＆応答合成
閾値を指定すると０件参照になったりするので注意。

In [15]:
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.postprocessor import SimilarityPostprocessor

# 後処理
# configure node postprocessors
# 類似度スコアにしきい値を設定してノードをフィルタリング
node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)]

# 応答合成
# configure response synthesizer
# response_modeを指定する。COMPACTはチャンク毎のLLM呼び出しを
# 最大プロンプト サイズ内に収まる限り多くのチャンクを詰め込む。
# また、Prompt Templateを与えデフォルトのPrompt Templateをカスタマイズ可能らしい。
response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT
)

##### 最終的な組み立て

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors
)

##### カスタマイズされたクエリの実行

###### チャンクが＋１された結果

In [17]:
# Either way we can now query the index
response = query_engine.query("What did the author do growing up?")
print(response)

The author worked on writing short stories and programming, particularly on an IBM 1401 computer in 9th grade using an early version of Fortran. Later on, the author got a microcomputer kit and started programming on a TRS-80, writing simple games and a word processor.


###### Kに対応したノード数になっているハズ

In [18]:
for node in response.source_nodes:
    print("score ", node.score)
    print("id_", node.id_)
    print("file_name", node.metadata["file_name"])
    # print("text", node.text) # テキストを確認したい場合、ココを実行して出力を確認
    print("------------------------------------------------------")

score  0.8108766833727953
id_ ef903a82-4999-448a-b007-452ab00100d5
file_name paul_graham_essay.txt
------------------------------------------------------
score  0.804940970839821
id_ 4aa4469e-4ece-4418-94ac-3a824149c490
file_name paul_graham_essay.txt
------------------------------------------------------
score  0.8017418372702448
id_ 79ed1d41-9d29-424d-982f-c33a4358bd0d
file_name paul_graham_essay.txt
------------------------------------------------------


### Evaluation
[共通](#共通)で説明したLoading、Indexing、Storing、Queryingの続き。

#### 準備

##### クエリを固定し評価用LLMを生成

In [19]:
query = "What did the author do growing up?"
llm = OpenAI(model="gpt-4", temperature=0.0)

##### 非同期実行のおまじない

In [20]:
import nest_asyncio
nest_asyncio.apply()

#### 関連評価
リクエストとレスポンスの関連を評価

In [21]:
from llama_index.core.evaluation import RelevancyEvaluator

# define evaluator
evaluator = RelevancyEvaluator(llm=llm)

# query index
query_engine = index.as_query_engine()
response = query_engine.query(query)

# evaluate response
relevancy_eval_result = evaluator.evaluate_response(query=query, response=response)
print("query: "           + str(relevancy_eval_result.query))
print("response: "        + str(relevancy_eval_result.response))
print("score: "           + str(relevancy_eval_result.score))
print("passing: "         + str(relevancy_eval_result.passing))         # バイナリ評価結果（合格か不合格か）	
print("feedback: "        + str(relevancy_eval_result.feedback))        # フィードバックまたは回答の理由
print("invalid_result: "  + str(relevancy_eval_result.invalid_result))  # 評価結果が無効かどうか。	
print("invalid_reason: "  + str(relevancy_eval_result.invalid_reason))  # 無効な評価の理由。
print("pairwise_source: " + str(relevancy_eval_result.pairwise_source)) # 

query: What did the author do growing up?
response: The author worked on writing short stories and programming, particularly on an IBM 1401 in 9th grade using an early version of Fortran. Later, with the advent of microcomputers, the author began programming on a TRS-80 and wrote simple games, a rocket prediction program, and a word processor.
score: 1.0
passing: True
feedback: YES
invalid_result: False
invalid_reason: None
pairwise_source: None


##### contexts
出力させると長いので概要だけ説明すると、  
質問（"What did the author do growing up?"）  
に関係するRAGのチャンクが抜き出されている。

In [ ]:
#print("contexts: " + str(relevancy_eval_result.contexts))

#### 回答評価
応答は（RAGによって）取得したコンテキストと一致しているか？

##### 実際の実行

In [23]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)

# query index
query_engine = index.as_query_engine()
response = query_engine.query(query)

# evaluate response
response_eval_result = evaluator.evaluate_response(response=response)
print("query: "           + str(response_eval_result.query))
print("response: "        + str(response_eval_result.response))
print("score: "           + str(response_eval_result.score))
print("passing: "         + str(response_eval_result.passing))         # バイナリ評価結果（合格か不合格か）	
print("feedback: "        + str(response_eval_result.feedback))        # フィードバックまたは回答の理由
print("invalid_result: "  + str(response_eval_result.invalid_result))  # 評価結果が無効かどうか。	
print("invalid_reason: "  + str(response_eval_result.invalid_reason))  # 無効な評価の理由。
print("pairwise_source: " + str(response_eval_result.pairwise_source)) # 

query: None
response: The author worked on writing short stories and programming, particularly on an IBM 1401 in 9th grade using an early version of Fortran. Later, the author transitioned to working with microcomputers, building simple games and a word processor on a TRS-80.
score: 1.0
passing: True
feedback: YES
invalid_result: False
invalid_reason: None
pairwise_source: None


##### contexts
出力させると長いので概要だけ説明すると、  
質問（"What did the author do growing up?"）  
に関係するRAGのチャンクが抜き出されている。

In [ ]:
#print("contexts: " + str(response_eval_result.contexts))

#### 検索評価
RAGによって取得されたソースはクエリに関連しているか？（先に使用した `retriever` を再利用）

In [25]:
from llama_index.core.evaluation import RetrieverEvaluator

# define evaluator
# hit_rate: 取得した上位k個のコンテキスト内に正しい答えが含まれている割合を計算
# mrr: 応答のリストを正解確率順に並べたプロセスを評価する統計的尺度（平均逆順位）
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

# query index
retrieval_eval_result = retriever_evaluator.evaluate(
    query=query,
    expected_ids=["node_id1", "node_id2"]
)

# evaluate response
response_eval_result = evaluator.evaluate_response(response=response)
print("query: "           + str(retrieval_eval_result.query))
print("expected_ids: "    + str(retrieval_eval_result.expected_ids))
print("expected_texts: "  + str(retrieval_eval_result.expected_texts))
print("retrieved_ids: "   + str(retrieval_eval_result.retrieved_ids))
print("mode: "            + str(retrieval_eval_result.mode))
print("metric_dict: "     + str(retrieval_eval_result.metric_dict))

query: What did the author do growing up?
expected_ids: ['node_id1', 'node_id2']
expected_texts: None
retrieved_ids: ['ef903a82-4999-448a-b007-452ab00100d5', '4aa4469e-4ece-4418-94ac-3a824149c490', '79ed1d41-9d29-424d-982f-c33a4358bd0d']
mode: RetrievalEvalMode.TEXT
metric_dict: {'mrr': RetrievalMetricResult(score=0.0, metadata={}), 'hit_rate': RetrievalMetricResult(score=0.0, metadata={})}


##### retrieved_texts
出力させると長いので概要だけ説明すると、  
質問（"What did the author do growing up?"）  
に関係するRAGのチャンクが抜き出されている。

In [ ]:
#print("retrieved_texts: " + str(retrieval_eval_result.retrieved_texts))

### StoringでChromaDBを使用

#### 追加のインストレーション3

```bash
!pip install chromadb
!pip install llama-index-vector-stores-chroma
```

#### 追加のライブラリ読み込み3

In [27]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

#### パーツ毎に分解して実行

##### Loading

In [28]:
# load some documents
documents = SimpleDirectoryReader(DATA_DIR).load_data()

##### Settings
Vector Store の Storage Context に Chroma DB を使う

In [29]:
# initialize client, setting path to save data
db = chromadb.PersistentClient(path=CHROMA_DIR)

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

##### Indexing & Storing
Index Store の Storage Context にも Chroma DB を使うので、  
Vector Store の Storage Context を Index Store にも設定する。  
※ OpenAI・Ollamaを切り替える際は、Embeddingが異なるようなので、一度、PERSIST_DIRのIndexを削除する。

###### 初回

```Python
# create your index and save index to chromadb
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
```

###### 2回目以降

In [30]:
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

###### 追加

```Python
for doc in documents:
    index.insert(doc)
```

##### Querying

In [31]:
# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("What is the meaning of life?")
print(response)

The meaning of life is a philosophical question that has been debated for centuries by various thinkers and scholars.


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/seigi/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/seigi/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/seigi/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/seigi/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1894, in _run_once
    handle = self._ready.popleft()
IndexError: p